In [1]:

import io
import os
import time
import datetime
import json
import shutil
import numpy as np
import pandas as pd
# import folium # draw route on map
from folium import plugins
import datetime
from datetime import timedelta
from pathlib import Path
from scipy.io import arff
import folium  # draw route on map
from folium import plugins
from PIL import Image

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go
%matplotlib inline

print(np.__version__)
print(pd.__version__)

!python3 --version

path_plot = Path('plot') 

1.19.2
1.3.3
Python 3.8.10


## Plot Event profile

In [2]:
file = 'datasets/DORA/dora_meta.csv'
meta = pd.read_csv(file , index_col=0)
meta['timestamp_conv'] = meta['timestamp_conv'].astype('datetime64[s]')
meta.drop(['timestamp', 'gsen_timeTick', 'gyro_timeTick'], axis=1, inplace=True)

In [3]:
meta_vehicle_id  = meta.groupby(['vehicle_id', 'event_id']).id.count().reset_index().drop(['id'], axis=1)
meta_vehicle_id 

,vehicle_id,event_id
0,KA01AF2992,1
1,KA01AF2992,2
2,KA01AF2992,3
3,KA01AF2992,4
4,KA01AF2992,7
...,...,...
7967,MH04KF8187,1093
7968,MH04KF8187,1094
7969,MH04KF8187,1095
7970,MH04KF8187,1096


- def plot_event()

In [4]:

items = ['speed', 'heading', 'gsen', 'gyro']
figsize=(12, 5)

def plot_event(df_, items=items, savepath_:str=None):
    fig, axes = plt.subplots(len(items), 1, figsize=figsize, sharex=True)
    fig.suptitle(f'{suptitle}')
    
    for ax, item in zip(axes, items):
        df_.filter(regex=f'^{item}', axis=1).plot(ax=ax, grid=True)
        
    if savepath_:
        fig.savefig(savepath_, bbox_inches='tight')     
# plot_event()

- plot event profile  by vehicle model

In [6]:
temp = meta.reset_index()

for tuple_ in meta_vehicle_id.itertuples():

    idx, v_model_, event_id_ =  tuple_
    
    path_plot = Path(f'plot/{v_model_}')
    if not path_plot.exists():
        path_plot.mkdir(exist_ok=True, parents=True)
    
    temp_ = temp.query('vehicle_id == @v_model_ and event_id == @event_id_').reset_index()
    timestamp_ = temp.timestamp_conv.iloc[0] 

    file_ = f'{v_model_}_event{event_id_}'
    suptitle  = file_ + f'\n{timestamp_}' 
    savepath_ = path_plot/f'{file_}.png'
    plot_event(temp_, savepath_=savepath_)
    plt.close()

    info = f'[{idx+1:4d}/{len(meta_vehicle_id)}]({100*idx/len(meta_vehicle_id):5.2f}%)'
    info += f'| {v_model_}, {event_id_ }'
    print(info, end='\r')
    if idx==1 :break


- plot_map()

In [6]:
def plot_map(temp_, savepath_=None): 
 
    m = folium.Map(location=temp_[['latitude', 'longitude']].iloc[0], zoom_start=18, control_scale=True)
    folium.PolyLine(temp_[['latitude', 'longitude']], color='red', weight=5, opacity=0.7).add_to(m)
    start_point = temp_[['latitude', 'longitude']].iloc[0].tolist()
    end_point = temp_[['latitude', 'longitude']].iloc[-1].tolist()
    folium.Marker(location=start_point,
                    icon=plugins.BeautifyIcon(border_color='red',
                                            text_color='black',
                                            number=1,
                                            inner_icon_style='plane')).add_to(m)
    folium.Marker(location=end_point,
                    icon=plugins.BeautifyIcon(border_color='yellow',
                                            text_color='black',
                                            number=2,
                                            inner_icon_style='plane')).add_to(m)
    mmap = m._to_png()
    img = Image.open(io.BytesIO(mmap))
    if savepath_:
        img.save(savepath_)

In [10]:
temp = meta.reset_index()

for tuple_ in meta_vehicle_id.itertuples():

    idx, v_model_, event_id_ =  tuple_
    if v_model_ != 'KA01AF2992':
        continue
    
    path_plot = Path(f'plot/{v_model_}')
    if not path_plot.exists():
        path_plot.mkdir(exist_ok=True, parents=True)
    
    temp_ = temp.query('vehicle_id == @v_model_ and event_id == @event_id_').reset_index()
    timestamp_ = temp.timestamp_conv.iloc[0] 

    file_ = f'{v_model_}_event{event_id_}'
    suptitle  = file_ + f'\n{timestamp_}' 
    savepath_ = path_plot/f'{v_model_}_event{event_id_}_map.png'
    plot_map(temp_,  savepath_=savepath_)
    plt.close()

    info = f'[{idx+1:4d}/{len(meta_vehicle_id)}]({100*idx/len(meta_vehicle_id):5.2f}%)'
    info += f'| {v_model_}, {event_id_ }'
    print(info, end='\r')
    # if idx==1 :break